In [34]:
import numpy as np
import sys
import os
sys.path.append(os.path.abspath("src"))
from agent import Actor, Critic, ReplayBuffer, DDPGAgent
from env import WirelessEnv
import torch

In [35]:
state_dim = 2
action_dim = 1
max_action = 10
Agent = DDPGAgent(state_dim, action_dim, max_action,buffer_size=1000)
env = WirelessEnv(num_antennas=1, num_users=1)

In [36]:
env_state = env._get_state()
print(env_state)

{'users': [{'id': 0, 'pos': array([32.45600253, 83.1355024 ]), 'demand': 1}], 'antennas': [{'id': 0, 'pos': array([ 0, 50])}]}


In [37]:
demand = env_state['users'][0]['demand']
print(demand)

1


In [38]:
distance = np.linalg.norm(env_state['users'][0]["pos"] - env_state['antennas'][0]["pos"])
print(f"distance : {distance:.2f}")
state = [demand, distance]
action = Agent.select_action(state)
action = {i: action[0] for i in range(1)}
reward = env._compute_rewards(action)
print(reward)

distance : 46.38
{0: 0.211047551564497}


In [39]:
print([state, action, reward])

[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]


In [40]:
batch_size = 20
for i in range(1000):
    action = Agent.select_action(state)
    action = {i: action[0] for i in range(1)}
    reward = env._compute_rewards(action)
    if i%100 == 0:
        print([state, action, reward])
    Agent.buffer.add(state, action, reward, state)


[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]
[[1, 46.38268663090346], {0: 10.0}, {0: 0.211047551564497}]


In [41]:
Agent.buffer.size()

1000

In [42]:

# Convertir en float
action = next(iter(action.values()))  # Récupère la valeur 0.20834593
reward = next(iter(reward.values()))  # Récupère la valeur 0.0034467490459354814

# Transformer en tenseurs PyTorch
action = torch.FloatTensor([action])
reward = torch.FloatTensor([reward])

print(action)  # tensor([0.2083])
print(reward)  # tensor([0.0034])


tensor([10.])
tensor([0.2110])
